In [141]:
from bs4 import BeautifulSoup
import requests
import csv
import re
from requests_html import HTMLSession # https://github.com/psf/requests-html

On cherche par rapport au nom de famille, les noms de famille de naissance sont utilisés. Il faut donc regarder le lieu de naissance des enfants pour savoir si la famille à bouger (suivie le/la partenaire).

Il faudrait arriver à trouver l'ancêtre le plus loin pour un nom de famille, puis analyser sa descendance :
Pour chaque enfant : 
    - Si c'est un garcon ou une fille on récupère les informations nescessaires + récupération des informations de l'épouse/époux
    - Si c'est un garcon on analysera la descendance


Informations à récupérer pour chaque personne :
 - Genre
 - Nom
 - Prénom
 - Date naissance
 - Lieu de naissance
 - Date mort
 - Lieu de mort


Visualisation (folium):
 - Sur une carte qui s'actualise tous les 10 ans, on affiche les naissances et les morts aux endroits correspondants

Avec le lieu de naissance de toute la famille, on peut voir comment ils se sont déplacé au fil des ans.

Améliorations :

Utilisation du lieu de mort, du lieu de certains événements, des informations des époux/épouses




In [142]:
# set up 
!pip install selenium
!pip install bs4
!pip install webdriver_manager
!pip install requests


^C


You should consider upgrading via the 'C:\Users\zied\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\zied\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\zied\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\zied\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# initialisation

# vidéo tuto : https://www.youtube.com/watch?v=3fcKKZMFbyA 


data_test = [{"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"},{"nom":"MARTIN", "prénom":"Jacques", "genre": "M", "ddn": 1822, "ldn":"Lyon", "ddm": 1898, "ldm":"Paris"}]

print(data_test)
data_test = sorted(data_test, key=lambda d: d['ddn']) 
print(data_test)


[{'nom': 'MARTIN', 'prénom': 'Jean', 'genre': 'M', 'ddn': 1855, 'ldn': 'Paris', 'ddm': 1920, 'ldm': 'Paris'}, {'nom': 'MARTIN', 'prénom': 'Jacques', 'genre': 'M', 'ddn': 1822, 'ldn': 'Lyon', 'ddm': 1898, 'ldm': 'Paris'}]
[{'nom': 'MARTIN', 'prénom': 'Jacques', 'genre': 'M', 'ddn': 1822, 'ldn': 'Lyon', 'ddm': 1898, 'ldm': 'Paris'}, {'nom': 'MARTIN', 'prénom': 'Jean', 'genre': 'M', 'ddn': 1855, 'ldn': 'Paris', 'ddm': 1920, 'ldm': 'Paris'}]


In [ ]:
# Futur fonction à utiliser (pour saisie du nom prenom et création de l'url correspondante)
def traitementURL():
    nom, prenom=getNomPrenom()
    url = "https://www.geneanet.org/fonds/individus/?go=1&nom=" + nom + "&prenom=" + prenom + "&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10" 
    return url 

def getNomPrenom():
    nom = input("Saisir nom")
    prenom = input("Saisir prénom")
    return nom, prenom

def getAncetres():
    nom, prenom = getNomPrenom()
    # récupérer f&s

    # récupérer père et mère 
    
    return

In [ ]:
# Url de recherche de test
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&prenom=fran%C3%A7ois&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10"

In [ ]:
# Récupération du code html
def getSoup(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup

In [ ]:
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&page=11&prenom=fran%C3%A7ois&prenom_operateur=or&size=10&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint="

# Récupération du code html
soup = getSoup(url)

# Récupération de la div contenant toutes les urls en lien avec la recherche
resOnly = soup.find("div", {"id":"table-resultats"})
# print(resOnly)

allArbreUrl = []
# Récupération des urls
for a in resOnly.find_all('a'):
    try:
        if (re.search("^arbres_utilisateur", a["data-id-es"])):  #^arbres_utilisateur
            allArbreUrl.append(a["href"])
    except:
        ""
print(allArbreUrl) #Comporte l'url des pages utilisateurs si se sont des arbres utilisateurs
    # headline = a.text-large
    # print(headline)

['https://gw.geneanet.org/palyart?n=martin&oc=&p=francois', 'https://gw.geneanet.org/fduffaux?n=martin&oc=&p=francois', 'https://gw.geneanet.org/jmalsagne?n=martin&oc=1&p=francois', 'https://gw.geneanet.org/philmart35?n=martin&oc=&p=francois', 'https://gw.geneanet.org/odyss1?n=martin&oc=&p=francois', 'https://gw.geneanet.org/mnoel18?n=martin&oc=39&p=francois', 'https://gw.geneanet.org/erikobay?n=martin&oc=&p=francois+jean+francois+claude']


In [155]:
# Récupération des ascendants (remonter facilement à l'ancetre)
resAnc = soup.find("div", {"href":'marmi1?lang=fr&pz=mathieu&nz=martin&p=pierre&n=martin&oc=13"'})
print(resAnc)


None


In [169]:
# Récupération du code source de chaque url correspondant à une personne
# {"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"}
session = HTMLSession()
# for url in allArbreUrl[0]:
for i in range(1):
    url = "https://gw.geneanet.org/palyart?n=martin&oc=&p=francois"
    r = session.get(url)

    print("Code html")
    soup = BeautifulSoup(r.html.html, 'html.parser')
    print(url)
    # print(soup)
    # Récupération de l'html de l'ensemble des données d'une personne
    person_html = soup.find("div", {"id":"perso"})

    # Récupération des données concernant la personne (genre, nom, prenom)
    person_title = person_html.find("div", {"id":"person-title"})
    person_title = person_title.find("h1")

    genre = person_title.find("img")["title"]
    
    nomprenom = person_title.find_all("a")
    nom = nomprenom[0].text
    prenom = nomprenom[1].text
    print(genre)
    print(nom)
    print(prenom)

    # Récupération des données concernant la personne (ddm, ldm, ddm, ldm)
    person_data_perso = person_html.find("ul")
    
    date_naissance = person_data_perso.find("li").text
    print(date_naissance)

    #print(re.search("Né le .* -", date_naissance))

    # Récupération des parents 
    person_parent = person_html.find("div", {"id":"parents"})
    # Récupération époux/épouse/enfants
    person_union = person_html.find("div", {"id":"fiche_union"})
    print(res)




Code html
https://gw.geneanet.org/palyart?n=martin&oc=&p=francois
H
François
MARTIN
Né le 4 septembre 1778 - Rochereaux - Saint-Cybardeaux, 16312, Charente, Poitou-Charentes, France
<div class="page_max row" id="perso">
<div class="columns">
<!--  Portrait -->
<div class="row clearfix" id="person-title">
<div class="columns">
<h1 class="with_tabs name">
<img alt="H" src="images/male.png" title="H"/> 
<a href="erikobay?lang=fr&amp;pz=eric+jean+lucien&amp;nz=bayet&amp;m=P&amp;v=francois+jean+francois+claude">François (Jean François Claude)</a>
<a href="erikobay?lang=fr&amp;pz=eric+jean+lucien&amp;nz=bayet&amp;m=N&amp;v=martin">MARTIN</a>
</h1>
<div class="action-buttons not-printable">
<a class="button radius small left bgcolor show-for-large-up" href="erikobay?lang=fr&amp;pz=eric+jean+lucien&amp;nz=bayet&amp;p=francois+jean+francois+claude&amp;n=martin&amp;type=graph" style="text-decoration: none;">
<span class="svg-icon-tree-print-white">Imprimer son arbre</span>
</a>
</div>
</div>
</d